# 새 데이터 가져오기

# eda
## 사명 변경된 적 있는지
```
final_df = df[df.groupby('company_sn')['company_name'].transform('nunique') > 1].sort_values(by='job_idx')[['company_sn', 'company_name']].drop_duplicates()
final_df['name_count'] = final_df.groupby('company_sn').cumcount() + 1
final_df.pivot_table(columns='name_count', index='company_sn', values='company_name', aggfunc='first')
```

## 목록에 표시된 해시태그
- 하지만 각 공고마다 가지고 있는 태그 전체 내용이 아니다.
```
df2 = df[['job_idx','job_sectors']]
df2.explode('job_sectors').groupby('job_sectors').size().sort_values(ascending=False).to_csv('job_sectors.csv')
df.explode('job_sectors').groupby('job_sectors').get_group('크롤링')

## jobsector 데이터가 조회할때마다 달라지는지 확인 필요
- 다섯개 이상 선택해도 리스트에 표시되지 않음
- 지정된 우선순위 순서로 표시되는지 아니면 조회할때마다 무작위로 5개 선택하는지 확인 필요

# 구글 시트에서 이전 시트 가져오기

import gspread
from gspread_dataframe import get_as_dataframe, set_with_dataframe

gc = gspread.service_account()

sh=gc.open_by_key('1vkZ7FF1bgACwXKxkmSNemmSNiti4Bd2M177w-8H80ww').sheet1




# modified          datetime64[ns]
# resistered        datetime64[ns]
# update            datetime64[ns]
# df_old의 세가지 열 데이터타입 변경


df_old = get_as_dataframe(sh, evaluate_formulas=False)
df_old.dropna(subset=['job_idx'], inplace=True)
df_old['job_idx'] = df_old['job_idx'].astype(int)
df_old.set_index('job_idx', inplace=True)
df_old['is_headhunting'] = df_old['is_headhunting'].astype(bool)
df_old["modified"] = pd.to_datetime(df_old["modified"])
df_old["resistered"] = pd.to_datetime(df_old["resistered"])
df_old["update"] = pd.to_datetime(df_old["update"])
df_old

# 새시트, 이전시트 병합
- jobsector 열 리스트 확장
- 제목 변경되는 경우 특이 케이스로 보고 따로 표시
- 수정일 : 새 시트 우선
- 작성일 : 이전 시트 우선, 원래 없다가 새로 생겨나는 경우 없을듯
- update 열 : 마지막으로 조회한 시간, 새 시트 우선
- 어떻게 합치는게 세련된 방법인지 모르겠다

- old에서 안쓰는 열은 드롭? 안돼 그러면 아우터가 아니니까

- 작성일 관측 불가면 어떻게 할지?

# Merge the dataframes
merged_df = pd.merge(df_old, df_new, how='outer', on=['job_idx'])

# Apply conditions to overlapping parts
# modified new 우선, resistered old 우선, update new 우선

merged_df["resistered"] = merged_df["resistered_x"].fillna(merged_df["resistered_y"])
merged_df["modified"] = merged_df["modified_x"].fillna(merged_df["modified_y"])
merged_df["update"] = merged_df["update_y"].fillna(merged_df["update_x"])

# Drop the columns
merged_df = merged_df.drop(columns=["resistered_x", "resistered_y", "modified_x", "modified_y", "update_x", "update_y"])
merged_df

# 새 시트로 덮어쓰기

# 키워드 처리
- job_sector 열에 키워드 들어있다.
- 키워드는 리스트 타입으로
- 각 공고마다 5개 이상의 키워드 가질 수 있다.
- 공고 목록에 표시되는 건 5개의 키워드 뿐
- 키워드로 목록을 새로 조회할 수 있다.
- 키워드로 목록을 조회할 때, 해당 목록의 모든 공고가 그 키워드를 가지고 있다고 등록해줘야함
- 자동차 라는 키워드로 목록 조회
- 그 목록을 1차 처리
- 키워드 리스트를 별도 테이블로 분리하고, 모든 공고에 검색 키워드 추가


전산총무 : 415
자동차 : 2218


In [1]:
import pandas as pd
import pickle

In [2]:
# Specify the file path of the pickle file
file_path = 'df_key.pickle'

# Load the pickle file
with open(file_path, 'rb') as file:
    df_key = pickle.load(file)


In [3]:
# Specify the file path of the pickle file
file_path = 'df_combined.pickle'

# Load the pickle file into df_old
with open(file_path, 'rb') as file:
    df_job = pickle.load(file)


In [4]:
df_job.shape

(5892, 14)

In [5]:
df_key.groupby('job_idx').size().sort_values(ascending=False)


job_idx
47868913    12
47849310    12
47913533    11
47871066    11
47870574    11
            ..
47910706     1
47783072     1
47879474     1
47874889     1
47918805     1
Length: 5892, dtype: int64

In [12]:
# 'job_sectors' 열에 대해 원-핫 인코딩 수행
one_hot_encoded_df = pd.get_dummies(filtered_job_idxs, columns=['job_sectors'], prefix='', prefix_sep='')

# 원-핫 인코딩 결과 합치기 (같은 job_idx에 대해 여러 행이 있을 경우)
final_df = one_hot_encoded_df.groupby('job_idx').max().reset_index()

final_df

,job_idx,4대보험,AI(인공지능),ASP,ASP.NET,C#,ERP,FAE,H/W,IR/공시,...,자금관리,자동차,자동차도장,자재관리,전산총무,전산회계,정보보안,정보통신,채용담당자,총무
0,47841638,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,True,True
1,47842859,False,False,False,False,False,False,False,False,False,...,False,True,False,True,False,False,False,False,False,True
2,47848218,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,False
3,47851102,False,True,True,True,False,True,False,False,False,...,False,True,False,False,True,False,False,False,False,False
4,47856455,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,True,False,False,False
5,47862010,False,False,False,False,False,False,False,False,False,...,True,True,False,False,True,False,False,False,False,True
6,47866742,False,False,False,False,False,False,False,False,False,...,False,True,True,False,False,False,False,False,False,True
7,47867867,False,False,False,False,False,False,False,False,False,...,False,True,False,False,False,False,False,False,False,True
8,47869810,False,False,False,False,False,True,False,True,False,...,False,True,False,False,False,False,False,False,False,False
9,47871493,False,False,False,False,False,False,False,False,False,...,False,True,False,False,True,False,True,False,False,False


In [13]:
final_df.set_index('job_idx', inplace=True)
final_df.sum().sort_values(ascending=False)

자동차            17
경영지원           17
전산총무            6
총무              5
유지보수            5
정보보안            4
보안컨설팅           3
웹개발             3
전산회계            3
SE(시스템엔지니어)     3
QA/테스터          3
S/W             2
문서작성            2
계산서발행           2
IR/공시           2
H/W             2
ERP             2
자동차도장           1
원가회계            1
앱개발             1
열처리             1
영업관리            1
영업직             1
외관검사원           1
원가관리            1
정보통신            1
채용담당자           1
자재관리            1
인터페이스           1
임베디드            1
자금관리            1
안전보건관리자         1
알고리즘            1
4대보험            1
안전관리자           1
기술영업            1
ASP             1
ASP.NET         1
C#              1
FAE             1
IT컨설팅           1
Java            1
MSSQL           1
관리회계            1
교육행정            1
네트워크            1
시설관리            1
더존              1
백엔드/서버개발        1
비품관리            1
사무보조            1
사무직             1
AI(인공지능)        1
생산관리            1
서무              1
손익관리      

In [2]:
from saramin import *

In [3]:
df_combined, df_key = load_dataframes()
data = fetch_job_data(cat_mcls=2,pages=5)
df_new = process_job_postings(data, 'IT개발·데이터')
df_combined = update_combined_dataframe(df_combined, df_new)
df_key = update_keyword_dataframe(df_key, df_new)
save_dataframes(df_combined, df_key)

The number of columns is 14.


In [6]:
df_combined, df_key = load_dataframes()


In [7]:
df_combined

,career_info,company_name,company_sn,deadline,education_info,is_headhunting,job_link,job_sectors,job_title,modified,resistered,support_date,update,work_place
job_idx,,,,,,,,,,,,,,
46367962,신입 · 경력 · 정규직 외,(주)한양기술공업,WHlqeXZqTWRZVWFlL2ZkQk4zVFA4dz09,233일 전 등록,고졸↑,False,https://www.saramin.co.kr/zf_user/jobs/relay/v...,"[총무, 문서작성, 비품관리, 서류관리, 시설관리, 전산총무]",㈜한양기술공업 상반기 각 부문 수시채용 공고,NaT,2023-08-16 17:39:36.554691,~05.21(화),2024-04-05 17:39:36.554691,경기 화성시 외
47327498,신입 · 정규직,(주)아모텍,ckdXSlNOd3BXV2FJcjRmeHdqNXpZZz09,88일 전 등록,"대학(2,3년)↑",False,https://www.saramin.co.kr/zf_user/jobs/relay/v...,"[기술지원, 데이터엔지니어, 백엔드/서버개발, 유지보수, 정보보안, 전산총무]",(주)아모텍 각 부문별 신입사원 모집,NaT,2024-01-08 17:39:36.552690,~04.30(화),2024-04-05 17:39:36.552690,서울 서초구 외
47328813,신입 · 정규직,(주)아모텍,ckdXSlNOd3BXV2FJcjRmeHdqNXpZZz09,87일 전 등록,"대학(2,3년)↑",False,https://www.saramin.co.kr/zf_user/jobs/relay/v...,"[기술지원, 데이터엔지니어, 백엔드/서버개발, 유지보수, 정보보안, 전산총무]",(주)아모텍 각 부문별 경력사원 모집,NaT,2024-01-09 17:39:36.549694,~04.30(화),2024-04-05 17:39:36.549694,서울 서초구 외
47447368,신입 · 경력 · 정규직 외,이에이트(주),OElPTVpmV0pLL0ZtVkl3RU5Cc3lkZz09,71일 전 등록,학력무관,False,https://www.saramin.co.kr/zf_user/jobs/relay/v...,"[백엔드/서버개발, 보안컨설팅, 앱개발, 정보보안, 프론트엔드, 전산총무]",이에이트 신입 및 경력 채용,NaT,2024-01-25 17:39:36.546694,~05.31(금),2024-04-05 17:39:36.546694,서울 송파구
47448498,신입 · 경력 · 정규직,교육지대(주),RlNrUnZiMVRBQ2ROVlN3Nlg4dVZBUT09,71일 전 등록,학력무관,False,https://www.saramin.co.kr/zf_user/jobs/relay/v...,"[유지보수, 총무, 문서작성, 비품관리, 사내행사, 전산총무]",[교육지대] 경영지원실 총무 채용,NaT,2024-01-25 17:39:36.543691,D-3,2024-04-05 17:39:36.543691,서울 구로구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47942797,경력무관 · 계약직 외,(주)서울커뮤니케이션,RFNWLytMQmMvN0EwQ0NJQS8vbHY1Zz09,17분 전 등록,고졸↑,False,https://www.saramin.co.kr/zf_user/jobs/relay/v...,"[사무직, 서무, 총무, 문서작성, 비품관리, 경영지원]",[종로/단순사무] 교직원 복지몰 운영 업무지원,NaT,2024-04-05 17:22:45.201778,~04.15(월),2024-04-05 17:39:45.201778,서울 종로구 외
47942824,경력 2년↑ · 계약직,에이치알서비스(주),cFZUeEoyRnQydThrRjZtb1kyRC9Mdz09,15분 전 등록,대학교(4년)↑,False,https://www.saramin.co.kr/zf_user/jobs/relay/v...,"[사무직, 서무, 내방객응대, 문서작성, 사무행정, 경영지원]",일본계기업 사무 채용 (수입/발주/영업지원),NaT,2024-04-05 17:24:45.199777,~05.05(일),2024-04-05 17:39:45.199777,서울 용산구
47942830,신입 · 경력 · 정규직,(주)IEN한창,STZPZ2g2Z2xFUllqRk1aVC9VU1RmZz09,14분 전 등록,학력무관,False,https://www.saramin.co.kr/zf_user/jobs/relay/v...,"[총무, 경영지원, 경영지원]",경영지원실 직원모집,NaT,2024-04-05 17:25:45.196778,~05.05(일),2024-04-05 17:39:45.196778,부산 기장군


In [9]:
# 필터링할 키워드 리스트
keywords = ['전산총무', 'IT개발·데이터']  

# 각 job_idx에 대해 필요한 키워드가 모두 포함되어 있는지 확인
def check_keywords(group):
    sectors = set(group['job_sectors'])
    return all(keyword in sectors for keyword in keywords)

# 'job_idx'로 그룹화하고, 각 그룹에 대해 check_keywords 함수를 적용하여 필터링
filtered_job_idxs = df_key.groupby('job_idx').filter(check_keywords)

filtered_job_idxs

# 각 그룹별로 job_sector를 리스트로 변환
s_idx = filtered_job_idxs.groupby('job_idx').apply(lambda group: list(group['job_sectors']))

df_job_filtered = df_combined.loc[s_idx.index]
df_job_filtered['job_sectors'] = s_idx.values

C:\Users\sol\AppData\Local\Temp\ipykernel_13924\2253516877.py:15: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  s_idx = filtered_job_idxs.groupby('job_idx').apply(lambda group: list(group['job_sectors']))


In [10]:
df_job_filtered

,career_info,company_name,company_sn,deadline,education_info,is_headhunting,job_link,job_sectors,job_title,modified,resistered,support_date,update,work_place
job_idx,,,,,,,,,,,,,,
47871493,신입 · 경력 · 정규직,(주)에코앤드림,YlJEMFNSYTYvdFhOaHhHWlcwYU14QT09,7시간 전 수정,고졸↑,False,https://www.saramin.co.kr/zf_user/jobs/relay/v...,"[보안컨설팅, 유지보수, 정보보안, IT컨설팅, 네트워크, 경영지원, 전산총무, I...",[코스닥] ㈜에코앤드림 2024년 부문별 신입/경력 채용,2024-04-05 10:39:53.286239,2024-04-05 11:35:48.328468,~04.30(화),2024-04-05 17:39:53.286239,충북 청주시 흥덕구 외
47873125,신입 · 경력 · 정규직 외,(주)알피바이오,dlVsalhjYU9wYmpjRWpZOVF5ZktNZz09,9일 전 등록,학력무관,False,https://www.saramin.co.kr/zf_user/jobs/relay/v...,"[기술지원, 보안컨설팅, 유지보수, 정보보안, IT컨설팅, 경영지원, 전산총무, I...",알피바이오 신입/경력 채용,NaT,2024-03-27 17:39:53.187566,~04.26(금),2024-04-05 17:39:53.187566,서울 강남구 외
47873384,신입 · 경력 · 정규직,(주)대승,OXZBeXlMVGxJbTB1bVFSZFFoNEJYUT09,9일 전 등록,"대학(2,3년)↑",False,https://www.saramin.co.kr/zf_user/jobs/relay/v...,"[유지보수, ERP, 감사, 내부감사, 외부감사, 전산총무, IT개발·데이터]",각 부문별 경력 및 신입사원 모집,NaT,2024-03-27 17:39:33.885149,~04.14(일),2024-04-05 17:39:33.885149,경기 평택시 외
47873539,경력무관 · 파견직,템프인(주),SEhKME5OVHRaRDN2N1d0QTB0eGMwZz09,9일 전 등록,대학교(4년)↑,False,https://www.saramin.co.kr/zf_user/jobs/relay/v...,"[기술지원, 유지보수, 네트워크, 인프라, 정보통신, 전산총무, IT개발·데이터]",[강남/월250~300만/정규전환]유명기업 기술지원(S/W분야),NaT,2024-03-27 17:39:33.883149,~04.26(금),2024-04-05 17:39:33.883149,서울 강남구
47875147,신입 · 경력 · 계약직,서린정보기술(주),ZTdjSmlCbWNNSmovRFp1cHRIR09HQT09,8시간 전 수정,"대학(2,3년)↑",False,https://www.saramin.co.kr/zf_user/jobs/relay/v...,"[기술지원, 유지보수, 총무, 문서작성, 비품관리, 전산총무, IT개발·데이터]",[서린정보기술] PC 및 주변기기 유지보수 담당자 채용,2024-04-05 09:39:33.866234,2024-04-05 09:35:47.680235,채용시,2024-04-05 17:39:33.866234,서울 강남구
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47939983,경력무관 · 파견직,(주)휴로넷,MGNLWTdKanpOTk01dWZKcXNXcVdkQT09,3시간 전 등록,학력무관,False,https://www.saramin.co.kr/zf_user/jobs/relay/v...,"[기술지원, 유지보수, 네트워크, 정보통신, S/W, 경영지원, 전산총무, IT개발...",SKT 신림사옥 5G품질측정및 관리 사무직채용(운전가능자),NaT,2024-04-05 14:39:45.394299,D-4,2024-04-05 17:39:45.394299,서울 관악구 외
47940694,경력무관 · 파견직,(주)헬로에이치알,ajJpNWVvWEdmcXNEWmdZbVRDY0pVQT09,2시간 전 등록,"대학(2,3년)↑",False,https://www.saramin.co.kr/zf_user/jobs/relay/v...,"[IT컨설팅, 사무직, 서무, 총무, 내방객응대, 경영지원, 전산총무, IT개발·데이터]",합정역/ 연3700수준 /복지上/그룹계열사 / IT총무,NaT,2024-04-05 15:39:45.320359,~05.05(일),2024-04-05 17:39:45.320359,서울 마포구
47941412,경력무관 · 정규직,알에프에이치아이씨(주),Q2RWLzhzV20rYmRoYXdNd3R6ZVMrdz09,1시간 전 등록,고졸↑,False,https://www.saramin.co.kr/zf_user/jobs/relay/v...,"[유지보수, 인프라, 비품관리, 자산관리, 전산총무, IT개발·데이터]",[신입/경력] IT 전산관리 팀원급 채용,NaT,2024-04-05 16:39:32.516846,~05.05(일),2024-04-05 17:39:32.516846,경기 과천시


In [11]:
# 구글시트로 보내는 함수

def send_to_google_sheet(df):
    import gspread
    from gspread_dataframe import get_as_dataframe, set_with_dataframe

    gc = gspread.service_account()

    sh=gc.open_by_key('1vkZ7FF1bgACwXKxkmSNemmSNiti4Bd2M177w-8H80ww').sheet1

    # Clear the contents of the sheet
    sh.clear()

    # Set the new dataframe in the sheet
    set_with_dataframe(sh, df, include_index=True, allow_formulas=False)


In [12]:
send_to_google_sheet(df_job_filtered)

# 키워드 테이블

In [7]:
# json 로드
import json

with open("searchPanelArgs.options.json", encoding="utf-8") as f:
    data_json = json.load(f)
data_cat = data_json['job_category']
import pandas as pd
df = pd.json_normalize(data_cat, record_path=["SMALL_LIST", "KEYWORD_LIST"])

# KEWD_CD_NO 열에 중복값 없는지 테스트
assert df['KEWD_CD_NO'].is_unique

import pickle

with open('scrape_rec_cat.pickle', 'wb') as file:
    pickle.dump(df, file)


# 테이블 로드

In [8]:
# 피클 파일 로드
import pickle

with open('scrape_rec_cat.pickle', 'rb') as file:
    df = pickle.load(file)


In [10]:
df

,MCLS_CD_NO,MCLS_CD_NM,SCLS_CD_NO,SCLS_CD_NM,KEWD_CD_NO,KEWD_CD_NM,COUNT
0,16,기획·전략,62,직무·직업,1624,게임기획,158
1,16,기획·전략,62,직무·직업,1625,경영기획,1644
2,16,기획·전략,62,직무·직업,1626,광고기획,1170
3,16,기획·전략,62,직무·직업,1627,교육기획,700
4,16,기획·전략,62,직무·직업,1628,기술기획,310
...,...,...,...,...,...,...,...
2152,20,공공·복지,74,전문분야,1967,장애인복지,91
2153,20,공공·복지,74,전문분야,1968,청소년복지,46
2154,20,공공·복지,74,전문분야,1969,호스피스,1
2155,20,공공·복지,74,전문분야,1970,EAP상담,16


In [9]:
df.groupby('MCLS_CD_NM').size()

MCLS_CD_NM
IT개발·데이터      248
건설·건축         192
고객상담·TM        32
공공·복지          44
교육            130
구매·자재·물류       50
금융·보험          59
기획·전략          69
디자인           148
마케팅·홍보·조사      66
미디어·문화·스포츠    131
상품기획·MD        70
생산            233
서비스           139
연구·R&D         60
영업·판매·무역      112
운전·운송·배송       57
의료            173
인사·노무·HRD      31
총무·법무·사무       48
회계·세무·재무       65
dtype: int64

In [15]:
df_it = df.groupby('MCLS_CD_NM').get_group('IT개발·데이터').sort_values('COUNT', ascending=False)

In [4]:
s = df.set_index('KEWD_CD_NM')['KEWD_CD_NO']

In [5]:
s['게임기획']

'1624'

In [6]:
s.get('게임기획')

'1624'

# eda

- 카테고리별 키워드 개수
- 너무 많이 요청 보내게 되는 것 아닌지
- 어쩌면 개별 공고를 열어서 키워드 알아내는게 더 빠를지도



In [16]:
from scrape_rec import *

In [ ]:
for keyword, code in KEYWORDS.items():


# 만약에 ERP만 조회한다면

In [18]:
s.get('ERP')

KEWD_CD_NM
ERP     372
ERP     149
ERP    1795
ERP    1113
Name: KEWD_CD_NO, dtype: object

In [19]:
df[df['KEWD_CD_NM']=='ERP']

,MCLS_CD_NO,MCLS_CD_NM,SCLS_CD_NO,SCLS_CD_NM,KEWD_CD_NO,KEWD_CD_NM,COUNT
188,3,회계·세무·재무,27,전문분야,372,ERP,1388
355,2,IT개발·데이터,24,전문분야,149,ERP,748
865,18,구매·자재·물류,68,전문분야,1795,ERP,1132
1301,11,생산,50,전문분야,1113,ERP,555


In [17]:
df_combined, df_key = load_dataframes()


In [20]:
data = fetch_job_data(1795)


ConnectTimeout: HTTPSConnectionPool(host='www.saramin.co.kr', port=443): Max retries exceeded with url: /zf_user/jobs/list/job-category?cat_kewd=1795&page=1&page_count=1000&sort=RD (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000026D7C315050>, 'Connection to www.saramin.co.kr timed out. (connect timeout=None)'))

In [ ]:
df_new = process_job_postings(data, keyword)
df_combined = update_combined_dataframe(df_combined, df_new)
df_key = update_keyword_dataframe(df_key, df_new)
save_dataframes(df_combined, df_key)